# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233678168460                   -0.50    8.0
  2   -7.250105310877       -1.78       -1.41    1.0
  3   -7.251308072113       -2.92       -2.07    4.0
  4   -7.251254966562   +   -4.27       -2.20    4.0
  5   -7.251314236100       -4.23       -2.47    2.0
  6   -7.251338119256       -4.62       -3.17    2.0
  7   -7.251338783196       -6.18       -4.09    2.0
  8   -7.251338793562       -7.98       -4.18    4.0
  9   -7.251338798305       -8.32       -4.69    2.0
 10   -7.251338798655       -9.46       -5.25    2.0
 11   -7.251338798702      -10.33       -5.84    3.0
 12   -7.251338798704      -11.69       -6.16    2.0
 13   -7.251338798705      -12.45       -6.82    3.0
 14   -7.251338798705      -13.62       -7.41    3.0
 15   -7.251338798705      -14.75       -7.73    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.053207820719386574
[ Info: Arnoldi iteration step 2: normres = 0.5140618324583824
[ Info: Arnoldi iteration step 3: normres = 0.9240512615542702
[ Info: Arnoldi iteration step 4: normres = 0.20912786811395268
[ Info: Arnoldi iteration step 5: normres = 0.3560664892455914
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.62e-02, 2.95e-02, 2.93e-01, 1.97e-01, 3.05e-02)
[ Info: Arnoldi iteration step 6: normres = 0.49500012907461954
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (6.69e-03, 1.21e-01, 4.53e-01, 1.30e-01, 8.52e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0868013014611168
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.36e-04, 3.58e-02, 7.72e-03, 5.32e-02, 4.92e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11579806629841129
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.72e